train 및 test는 DACON에서 직접 다운받으시길 바랍니다.
- 링크 : https://dacon.io/competitions/official/236176/data

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from datetime import datetime,timedelta
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from xgboost import XGBClassifier
import datetime
import holidays
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor, plot_importance
import random
import os
import xgboost
from sklearn.model_selection import TimeSeriesSplit
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor


In [30]:
def seed_everything(seed: int = 2024):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
seed_everything(2024)

# Pre-Processing

In [31]:
def pre_all(train, test):
    print(f"전처리 전 train 크기 : {train.shape}")
    print(f"전처리 전 test 크기 : {test.shape}")
    print("=================전처리 중=================")

    # 합쳐서 전처리하기
    train["timestamp"] = pd.to_datetime(train["timestamp"])
    test["timestamp"] = pd.to_datetime(test["timestamp"])
    df = pd.concat([train,test]).reset_index(drop = True)

    df.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'},inplace=True)

    #년/월/일 추가
    df['year']=df['timestamp'].dt.year
    df['month']=df['timestamp'].dt.month
    df['day']=df['timestamp'].dt.day

    #요일 추가
    df['week_day']=df['timestamp'].dt.weekday

    # 년-월 변수 추가 : year-month의 형태, 개월단위 누적값
    le = LabelEncoder()
    df["year_month"] = df["timestamp"].map(lambda x :str(x.year) + "-"+str(x.month))

    # 라벨 인코딩
    df["year_month"] = le.fit_transform(df["year_month"])


    # 주차 변수 추가
    df["week"] = df["timestamp"].map(lambda x: datetime.datetime(x.year, x.month, x.day).isocalendar()[1])

    # 주차 누적값
    week_list=[]
    for i in range(len(df['year'])) :
        if df['year'][i] == 2019 :
            week_list.append(int(df['week'][i]))
        elif df['year'][i] == 2020 :
            week_list.append(int(df['week'][i])+52)
        elif df['year'][i] == 2021 :
            week_list.append(int(df['week'][i])+52+53)
        elif df['year'][i] == 2022 :
            week_list.append(int(df['week'][i])+52+53+53)
        elif df['year'][i] == 2023 :
            week_list.append(int(df['week'][i])+52+53+53+52)
    df['week_num']= week_list

    # datetime 패키지에서 19년 12월 마지막주가 첫째주로 들어가는거 발견하여 수정
    df.loc[df['timestamp']=='2019-12-30','week_num']=52
    df.loc[df['timestamp']=='2019-12-31','week_num']=52


    # 공휴일 변수 추가
    def make_holi(x):
        kr_holi = holidays.KR()

        if x in kr_holi:
            return 1
        else:
            return 0
        
    df["holiday"] = df["timestamp"].map(lambda x : make_holi(x))
    

    # train, test 분리하기
    train = df[~df["price"].isnull()].sort_values("timestamp").reset_index(drop = True)
    test = df[df["price"].isnull()].sort_values("timestamp").reset_index(drop=True)


    print(f"전처리 후 train 크기 : {train.shape}")
    print(f"전처리 후 test 크기 : {test.shape}")

    return train, test

# 데이터 불러오기

In [32]:
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")
trade = pd.read_csv("dataset/international_trade.csv")
submission = pd.read_csv("dataset/sample_submission.csv")

In [33]:
# # 일자별 중복 개수 확인
train.groupby(["item",  "timestamp"]).count()

ID  corporation  location  supply(kg)  price(원/kg)
item timestamp                                                     
BC   2019-01-01   9            9         9           9            9
     2019-01-02   9            9         9           9            9
     2019-01-03   9            9         9           9            9
     2019-01-04   9            9         9           9            9
     2019-01-05   9            9         9           9            9
...              ..          ...       ...         ...          ...
TG   2023-02-27  10           10        10          10           10
     2023-02-28  10           10        10          10           10
     2023-03-01  10           10        10          10           10
     2023-03-02  10           10        10          10           10
     2023-03-03  10           10        10          10           10

[7615 rows x 5 columns]

In [34]:
# 품목별 중복일수 딕셔너리화
item_unique = train.groupby(["item",  "timestamp"]).count().reset_index()
item_cnt = {}
for item in train.item.unique():
    item_cnt[item] = int(item_unique[item_unique["item"] == item]["ID"].unique())

In [35]:
# 품목별 날짜의 중복 개수 확인 > TSCV진행시 test_size를 중복개수 * valid set개수로 지정한다.
item_cnt

{'TG': 10, 'CR': 7, 'CB': 5, 'RD': 8, 'BC': 9}

# TG TSCV

In [36]:
# 데이터 불러오기
# 공휴일이지만 안쉬는 날 제외하기
train_pre, test_pre = pre_all(train, test)

no_holi = list(train_pre[(train_pre["item"] =="TG") &(train_pre["holiday"]==1) & (train_pre["price"]!=0)].groupby("timestamp").count().reset_index()["timestamp"])
noholi_idx = train_pre[train_pre["timestamp"].isin(no_holi)]["holiday"].index
for idx in noholi_idx:
    train_pre.loc[idx, "holiday"] = 0

# train 및 test 시간 순서로 정렬하기
train_tg = train_pre[train_pre["item"] == "TG"].sort_values(by = ["timestamp"]).reset_index(drop= True)
test_tg = test_pre[test_pre["item"] == "TG"].sort_values(by = ["timestamp"]).reset_index(drop= True)

Xy = pd.get_dummies(train_tg, columns = [ "item","corporation","location"]).drop(columns = ["supply"])
answer_tg1 = pd.get_dummies(test_tg, columns = [ "item","corporation","location"]).drop(columns = ["timestamp","supply","price"])
print(f"train의 컬럼 : {Xy.columns}")
print(f"test의 컬럼 : {answer_tg1.columns}")
Xy["price"] = np.sqrt(Xy["price"])


전처리 전 train 크기 : (59397, 7)
전처리 전 test 크기 : (1092, 5)
=================전처리 중=================
전처리 후 train 크기 : (59397, 15)
전처리 후 test 크기 : (1092, 15)
train의 컬럼 : Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'item_TG', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'location_J', 'location_S'],
      dtype='object')
test의 컬럼 : Index(['ID', 'year', 'month', 'day', 'week_day', 'year_month', 'week',
       'week_num', 'holiday', 'item_TG', 'corporation_A', 'corporation_B',
       'corporation_C', 'corporation_D', 'corporation_E', 'location_J',
       'location_S'],
      dtype='object')


In [20]:
# XGboost의 전체 및 감귤만 RMSE CV 비교해보기
models = []

# rmse
tg_rmses = []

# cv 데이터 확인
datas = []

# catboost 파라미터
n_estimators =1000
lrs = 0.01
max_depths = 10
gamma = 0.5
l2_leaf_reg = 3

# valid set 일 수
n = 28

tscv = TimeSeriesSplit(test_size=item_cnt["TG"]*n, n_splits = 5)
for train_idx, test_idx in tscv.split(Xy):

    # 데이터 정리
    X_train = Xy.iloc[train_idx].drop(columns = ["timestamp", "price", "ID"])
    X_valid = Xy.iloc[test_idx].drop(columns = ["timestamp", "price", "ID"])

    y_train = Xy.iloc[train_idx]["price"]
    y_valid = Xy.iloc[test_idx]["price"]
    datas.append([X_train, X_valid])

    # # 모델 정의
    cat = CatBoostRegressor(random_state = 2024, 
                                n_estimators = n_estimators, 
                                learning_rate = lrs, 
                                depth = max_depths,
                                l2_leaf_reg = l2_leaf_reg,
                                metric_period = 1000)
    
    xgb = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.05, max_depth = 10)


    # voting
    vote_model = VotingRegressor(
        estimators =[("cat",cat), ("xgb", xgb)]
    )
    vote_model.fit(X_train, y_train)
    models.append(cat)

    pred = vote_model.predict(X_valid)
    for idx in range(len(pred)):
        if pred[idx]<0:
            pred[idx]=0
    
    # 전체 rmse 구해보기
    tmp_rmse = np.sqrt(mean_squared_error(np.power(y_valid,2), np.power(pred,2)))
    
    tg_rmses.append(tmp_rmse)
    print(tmp_rmse)
    print()


print(f"감귤의 평균 RMSE : {np.mean(tg_rmses)}")


0:	learn: 31.7632998	total: 10.9ms	remaining: 10.9s
999:	learn: 15.9979750	total: 8.37s	remaining: 0us
473.8305280993594

0:	learn: 31.5749434	total: 11.4ms	remaining: 11.4s
999:	learn: 15.8702506	total: 9.09s	remaining: 0us
325.8391403024061

0:	learn: 31.3827088	total: 12ms	remaining: 12s
999:	learn: 15.7279708	total: 9.17s	remaining: 0us
666.453726926735

0:	learn: 31.2115626	total: 14.3ms	remaining: 14.3s
999:	learn: 15.6618255	total: 9.43s	remaining: 0us
932.0528987795547

0:	learn: 31.0896384	total: 13.3ms	remaining: 13.3s
999:	learn: 15.5797961	total: 9.42s	remaining: 0us
1005.7948398087891

감귤의 평균 RMSE : 680.7942267833689


# TG 제외 TSCV

In [42]:
## 전처리
train_pre, test_pre = pre_all(train, test)

# 극 이상치 제거
tg_idx = train_pre[(train_pre["item"]=="TG") & (train_pre["price"]>20000)].index
rd_idx = train_pre[(train_pre["item"]=="RD") & (train_pre["price"]>5000)].index
bc_idx = train_pre[(train_pre["item"]=="BC") & (train_pre["price"]>8000)].index
cb_idx = train_pre[(train_pre["item"]=="CB") & (train_pre["price"]>2300)].index

train_pre.loc[tg_idx,"price"] = train_pre[(train_pre["item"]=="TG") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[rd_idx,"price"] = train_pre[(train_pre["item"]=="RD") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[bc_idx,"price"] = train_pre[(train_pre["item"]=="BC") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[cb_idx,"price"] = train_pre[(train_pre["item"]=="CB") & (train_pre["price"]!=0)]["price"].mean()


# 감귤이 아닌것
print(f"train의 컬럼 : {train_pre.columns}")
print(f"test의 컬럼 : {test_pre.columns}")

train_notg = train_pre[train_pre["item"] !="TG"]
test_notg = test_pre[test_pre["item"] != "TG"]


#인코딩
Xy = pd.get_dummies(train_notg.sort_values(by = ["timestamp"]).reset_index(drop=True).drop(columns = ["supply"]), columns = ["item","corporation","location"])
answer_notg = pd.get_dummies(test_notg.drop(columns = ["timestamp","supply","price"]), columns = [ "item","corporation","location"])
print(Xy.columns)


전처리 전 train 크기 : (59397, 7)
전처리 전 test 크기 : (1092, 5)
=================전처리 중=================
전처리 후 train 크기 : (59397, 15)
전처리 후 test 크기 : (1092, 15)
train의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')
test의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')
Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'item_BC', 'item_CB',
       'item_CR', 'item_RD', 'corporation_A', 'corporation_B', 'corporation_C',
       'corporation_D', 'corporation_E', 'corporation_F', 'location_J',
       'location_S'],
      dtype='object')


In [43]:
# 감귤이 제외된 품목의 TSCV 실행
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor
models = []

# rmse
notg_rmse = []

# cv 데이터 확인
datas = []

# 예측 및 
n_estimators =1000
lrs = 0.01
max_depths = 10
l2_leaf_reg = 3

# valid set 일 수
n = 28

tscv = TimeSeriesSplit(test_size=(item_cnt["CR"] + item_cnt["CB"] + item_cnt["RD"] + item_cnt["BC"])*n, n_splits = 5)
for train_idx, test_idx in tscv.split(Xy):

    # 데이터 정리
    X_train = Xy.iloc[train_idx].drop(columns = ["timestamp", "price", "ID"])
    X_valid = Xy.iloc[test_idx].drop(columns = ["timestamp", "price", "ID"])

    y_train = Xy.iloc[train_idx]["price"]
    y_valid = Xy.iloc[test_idx]["price"]
    datas.append([X_train, X_valid])

    # # 모델 정의
    cat = CatBoostRegressor(random_state = 2024, 
                                n_estimators = n_estimators, 
                                learning_rate = lrs, 
                                depth = max_depths,
                                l2_leaf_reg = l2_leaf_reg,
                                metric_period = 1000)
    
    xgb = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.05, max_depth = 10)

    # voting
    vote_model = VotingRegressor(
        estimators =[("cat",cat), ("xgb", xgb)]
    )

    vote_model.fit(X_train, y_train)
    models.append(cat)

    pred = vote_model.predict(X_valid)
    for idx in range(len(pred)):
        if pred[idx]<0:
            pred[idx]=0

    # 전체 rmse 구해보기
    tmp_rmse = np.sqrt(mean_squared_error(y_valid, pred))
    notg_rmse.append(tmp_rmse)

    print(tmp_rmse)
    print()


print(f"TG 제외의 평균 RMSE : {np.mean(notg_rmse)}")

#

0:	learn: 918.7448329	total: 19.8ms	remaining: 19.8s
999:	learn: 356.5699902	total: 11.8s	remaining: 0us
616.955248649581

0:	learn: 925.5982785	total: 16.7ms	remaining: 16.7s
999:	learn: 361.6470767	total: 13.6s	remaining: 0us
707.9763267651905

0:	learn: 924.2492202	total: 15.8ms	remaining: 15.8s
999:	learn: 364.4609246	total: 11.6s	remaining: 0us
707.3383814732375

0:	learn: 928.3240746	total: 15.1ms	remaining: 15.1s
999:	learn: 369.3291645	total: 11.7s	remaining: 0us
619.1679917731552

0:	learn: 930.3118570	total: 15.8ms	remaining: 15.8s
999:	learn: 373.2141521	total: 12s	remaining: 0us
472.0917603819723

TG 제외의 평균 RMSE : 624.7059418086272
